### Citations - points of inspiration for the whole project

The work of Mohapatra, Ahmed and Alencar has been of great inspiration to me (https://github.com/Nomiizz/KryptoOracle) during my time of writing code down, as well as countless stackoverflow posts to troubleshoot and surpass certain obstacles and TowardsDataScience pages to fetch for ideas.

## Imports

In [2]:
import os
from time import sleep
import json
import pandas as pd
import io
from tqdm import tqdm
from twython import Twython
import numpy as np
import datetime
import time
from datetime import datetime
from datetime import timedelta

## Access

In [2]:
path = os.getcwd()
path
tweets_notclean = 'Data Collection/twitter/BTC/bitcoin_tweets_notclean.csv' # The file where we will store the tweets before preprocessing them
query = '#bitcoin OR #BTC OR bitcoin OR BTC OR $BTC' # Our query to use when searching for tweets
tweets_notclean_nextid = 'Data Collection/twitter/BTC/bitcoin_tweets_notclean_week_nextid.csv' 

In [3]:
API_KEY ='gU1T2G85YKFZVNr0i9Mx6rTh1'  # We got these from our app on the Twitter Developer Dashboard
API_SECRET_KEY =  'Kl1TT1C0zY7NPnD1TF6XPS0nfZfY3dT1XZlb1CDBlzFIu9Qqhh' 
twitter = Twython(API_KEY, API_SECRET_KEY, oauth_version=2) # We use version 2 because it allows us to get more tweets
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(API_KEY, access_token=ACCESS_TOKEN)
twitter.get_application_rate_limit_status()['resources']['search'] # We check to see what the rate limit is

{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1618989538}}

## Tweet Extraction Stream

We use the Twitter Developer account that we obtained after applying for it to make a stream that extracts tweets that contain one (or more) of the terms 'bitcoin' , '#BTC' , '#bitcoin', 'BTC', '$BTC'

In [4]:
iterations = 450
data = {"statuses": []} # Dictionary to store the data values
next_id = "" # This will indicate the id of the next tweet
since_id= "" # This will indicate the flag, ids that are greater than the flag (more recent) will be returned

with open(tweets_notclean_nextid,"a+", encoding='utf-8') as f: 
    # a+ : Open for reading and writing. The file is created if it does not exist. 
    # The stream is positioned at the end of the file.
    
    if not next_id and not since_id: #if there is no next_id and no since_id:
        f.write("ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt\n") 
        # The first row of our csv will be the column names
        
    while(True):
        twitter = Twython(API_KEY, access_token=ACCESS_TOKEN)
        size = 0 # Indicates the size so that we know when we stop
        
        for i in tqdm(range(iterations)): # Repeat for 450 times (our rate limit)
            
            if not next_id:
                data = twitter.search(q=query, lang='en', result_type='recent', count="100",tweet_mode='extended',since_id=since_id) 
                # At the start (next_id is blank) we search for the most recent tweets (100 of them per iteration) and set the since_id to blank for the first iteration
            elif since_id:
                data["statuses"].extend(twitter.search(q=query, lang='en', result_type='mixed', count="100",max_id=next_id,tweet_mode='extended')["statuses"])
                # We set the max_id to be the id of  the last status, so that we can collect tweets with ids older than the max_id
            else: # If next_id exists and there is no since_id
                data["statuses"].extend(twitter.search(q=query, lang='en', result_type='mixed', count="100", max_id=next_id,tweet_mode='extended')["statuses"])
                # We set the max_id to be the id of  the last status, so that we can collect tweets with ids older than the max_id
                
            if len(data["statuses"]) > 1: # If we have more than one statuses collected we set next_id to be the id of the last status
                next_id = data["statuses"][len(data["statuses"]) - 1]['id']
            if size + 1 == len(data["statuses"]): # If we can't collect any more statuses (we reached our limit for this iteration ie 100 tweets)
                break
            else:
                size = len(data["statuses"])
            
        print('Retrieved {0}, waiting for 15 minutes until next queries'.format(len(data["statuses"])))
        
        d = pd.DataFrame([[s["id"], s["full_text"].replace('\n','').replace('\r',''), s["user"]["name"], s["user"]["followers_count"], s["retweet_count"], s["favorite_count"], s["created_at"]] for s in data["statuses"]], columns=('ID', 'Text', 'UserName', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
        # We make a pandas dataframe to store our tweets, where we replace the newline characters and the tab characters with a space on the texts
        d.to_csv(f, mode='a', encoding='utf-8',index=False,header=False)
        if size + 1 == len(data["statuses"]):
            print('No more new tweets, stopping...')
            break 
        data["statuses"] = []
        
        sleep(910) # We sleep for 15 minutes so that the rate limit gets renewed after we exceed it

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [01:49<00:00,  4.11it/s]


Retrieved 0, waiting for 15 minutes until next queries


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [01:51<00:00,  4.05it/s]


Retrieved 0, waiting for 15 minutes until next queries


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [01:46<00:00,  4.21it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [01:51<00:00,  4.05it/s]


Retrieved 0, waiting for 15 minutes until next queries


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [01:49<00:00,  4.10it/s]


Retrieved 0, waiting for 15 minutes until next queries


KeyboardInterrupt: 

In [ ]:
#week 1
# 1.543.263 tweets collected, 11h 2min 13sec
# start date Sat Mar 13 11:47:11 +0000 2021, end date Sat Mar 06 03:47:49 +0000 2021
# 7 days + 8 hours back

#week 2
# 1.875.727 tweets collected
# start date Sun Mar 28 8:26, end date Sat Mar 20 18h:17
# 7 days + 14 hours


#week 3 
# 1.546.735 tweets collected, 11h 12min 38sec
# start date Sat Apr 03 09:01:12 +0000 2021, end date Fri Mar 26 15:00:03 +0000 2021
# pige 7 meres + 18 wres pisw

#week4 
# 1.550.457 tweets collected, 11h 9min 38sec
# start date Sat Apr 10 09:01:00 +0000 2021, end date Sat Apr 03 01:00:43 +0000 2021
# pige 7 meres + 8 wres pisw

#there is a 1.5 days gap between week 4 and week 5
#week5 
# 2.093.415  tweets collected
#start date Mon Apr 19 21:39:51 +0000 2021, end date Sun Apr 11 19:49:24 +0000 2021


#total 7.066.334


## Creating a function that rounds time 

I want to make code that converts all the elements of the CreatedAt column from str to datetime and then rounds 
 each element to the hour or to the 15 minute mark (depends on the time window we decide to get). We also use this function in the Prediction Notebooks

In [4]:
tweets1 = pd.read_csv(r"C:\Users\Fivos\Desktop\Πτυχιακή\Data Collection\twitter\BTC\bitcoin_tweets_notclean_Tue20-4_14h55m_Sun18-4_14h38m.csv")
tweets1

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
0,1384521088089079808,$CELOLatest add into my portfolio is #celo.We ...,Siho,146,0,0,Tue Apr 20 14:55:25 +0000 2021
1,1384521088038752257,*3IQ DIGITAL ASSET MGMT TO LIST BITCOIN FUND O...,sellvolbuytesla,603,0,0,Tue Apr 20 14:55:25 +0000 2021
2,1384521087912927234,Bitcoin keeps getting rejected from many trend...,Crypto Bible,36798,0,0,Tue Apr 20 14:55:25 +0000 2021
3,1384521087057203201,RT @michael_saylor: #Bitcoin on Venmo for 70 m...,cjglory,525,1141,0,Tue Apr 20 14:55:24 +0000 2021
4,1384521086369476609,"Tue Apr 20 16:54:13 2021 (30:40)USD : 55,462.7...",Block Watcher,909,0,0,Tue Apr 20 14:55:24 +0000 2021
...,...,...,...,...,...,...,...
539205,1383792115134599179,RT @MocktailSwap: Say Hello to our MocktailSwa...,Justice,39,4038,0,Sun Apr 18 14:38:44 +0000 2021
539206,1383792114832613376,RT @justinsuntron: #TRX now is the most active...,Ojo Ibukunoluwa,31,2195,0,Sun Apr 18 14:38:44 +0000 2021
539207,1383792114153050126,RT @michael_saylor: In preparation for my upco...,On The Hunt #BuyBitcoin,265,899,0,Sun Apr 18 14:38:44 +0000 2021
539208,1383792113347743749,RT @Udedibia: FOX to the moon🚀 #FOXFINANCE #FO...,Rushikesh🖤,2377,33,0,Sun Apr 18 14:38:44 +0000 2021


In [5]:
t1 = tweets1['CreatedAt'][0]
t1

'Tue Apr 20 14:55:25 +0000 2021'

In [8]:
new_t1 = datetime.strptime(t1,'%a %b %d %H:%M:%S +0000 %Y')
print(new_t1)

2021-04-20 14:55:25


In [9]:
type(new_t1)

datetime.datetime

In [10]:
def roundTime(dt=None, roundTo=60):
   """Round a datetime object to any time lapse in seconds
   dt : datetime.datetime object, default now.
   roundTo : Closest number of seconds to round to, default 1 minute.
   Author: Thierry Husson 2012 - Use it as you want but don't blame me.
   """
   if dt == None : dt = datetime.datetime.now()
   seconds = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + timedelta(0,rounding-seconds,-dt.microsecond)


In [11]:
roundedt1 = roundTime(new_t1,60*15)
#we round it to 15 minutes in this example
print(roundedt1)

2021-04-20 15:00:00


In [ ]:
tweets1['FlooredDatatime'] = 0
for i in tqdm(range(len(tweets1))):
    tweets1['FlooredDatatime'][i] = roundTime(datetime.strptime(tweets1['CreatedAt'][i],'%a %b %d %H:%M:%S +0000 %Y'),60*15)